In [ ]:
import os, sys
sys.path.insert(1, os.path.join(os.getcwd()  , '../..'))

In [13]:
# imports
import random
import multiprocessing as mp
from collections import defaultdict
from core import * 
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

In [14]:
possible_items = [
    TwoDimensionalItem(300, 400),
    TwoDimensionalItem(600, 300),
    TwoDimensionalItem(600, 400),
    TwoDimensionalItem(300, 200),
]

vehicle = Vehicle(
    compartments=[
        TwoDimensionalCompartment(800, 300),
        TwoDimensionalCompartment(800, 300),
        TwoDimensionalCompartment(800, 300),
        TwoDimensionalCompartment(800, 300),
        TwoDimensionalCompartment(800, 600),
        TwoDimensionalCompartment(800, 600),
    ]
)

# Define some problems

# Problem 1
nb_items_1 = 15
V_1 = list(range(0, nb_items_1 * 2 + 1))
random.seed(0)
items_1 = random.choices(possible_items, k=nb_items_1)
random.seed(0)
distance_matrix_1 = [
    [random.randint(1, 100) if i != 0 and j!=0 and i != j else 0 for i in V_1]
    for j in V_1
]
problem_1 = TwoDimensionalProblem(items_1, vehicle, distance_matrix_1)

# Problem 2
nb_items_2 = 15
V_2 = list(range(0, nb_items_2 * 2 + 1))
random.seed(100)
items_2 = random.choices(possible_items, k=nb_items_2)
random.seed(100)
distance_matrix_2 = [
    [random.randint(1, 100) if i != 0 and j!=0 and i != j else 0 for i in V_2]
    for j in V_2
]
problem_2 = TwoDimensionalProblem(items_2, vehicle, distance_matrix_2)

In [15]:
# Solve problem 1 greedily
solver = GreedySolver(problem_1)
# get some greedy solutions
nb_solutions = 100000
solutions = solver.solve(nb_solutions=nb_solutions)

In [16]:
# Randomly choose an initial solution
initial_solution = random.sample(solutions, 1)[0]

In [32]:
time_limit = 200
# Create LNS with relaxed accept
accept_criterion = SimulatedAnnealing(alpha=0.995, starting_temperature=100) # instead of 1.05
relaxed_LNS = LNS(problem_1, initial_solution, time_limit=time_limit)
relaxed_LNS.acceptance_criterion = accept_criterion

# Create LNS with default simulated annealing
normal_LNS = LNS(problem_1, initial_solution, time_limit=time_limit)

In [33]:
# run solvers
relaxed_LNS.search()
normal_LNS.search()

Terminating LNS in 969 iterations
Total time: 200.06 Best objective: 253 found in 601 iterations in 126.53 s

Terminating LNS in 954 iterations
Total time: 200.13 Best objective: 234 found in 500 iterations in 108.99 s



Solution(
	order=0, 8, 4, 15, 7, 6, 12, 10, 21, 25, 9, 3, 14, 24, 2, 27, 23, 1, 5, 30, 18, 16, 29, 11, 13, 26, 28, 22, 19, 20, 17, 
	step: 0 vertex: 0 	0, 0, 0, 0, 0, 0
	step: 1 vertex: 8 	0, 0, 0, 0, 300, 0
	step: 2 vertex: 4 	0, 0, 600, 0, 300, 0
	step: 3 vertex: 15 	0, 0, 600, 0, 300, 400
	step: 4 vertex: 7 	0, 0, 800, 0, 300, 400
	step: 5 vertex: 6 	600, 0, 800, 0, 300, 400
	step: 6 vertex: 12 	600, 0, 800, 0, 700, 400
	step: 7 vertex: 10 	600, 0, 800, 0, 700, 800
	step: 8 vertex: 21 	0, 0, 800, 0, 700, 800
	step: 9 vertex: 25 	0, 0, 800, 0, 700, 400
	step: 10 vertex: 9 	600, 0, 800, 0, 700, 400
	step: 11 vertex: 3 	600, 0, 800, 600, 700, 400
	step: 12 vertex: 14 	600, 200, 800, 600, 700, 400
	step: 13 vertex: 24 	0, 200, 800, 600, 700, 400
	step: 14 vertex: 2 	200, 200, 800, 600, 700, 400
	step: 15 vertex: 27 	200, 200, 800, 600, 300, 400
	step: 16 vertex: 23 	200, 200, 800, 600, 0, 400
	step: 17 vertex: 1 	400, 200, 800, 600, 0, 400
	step: 18 vertex: 5 	400, 200, 800, 600, 400, 4

In [27]:
def plot_solver_stats(solver, fig=None, name=""):
    fig = fig or make_subplots(
        rows=1,
        cols=2,
        shared_yaxes=True,
    )

    objectives = [sol.objective for sol in solver.solutions_cache]
    iterations = [sol.iteration for sol in solver.solutions_cache]
    time_found = [
        "{:.2f}".format(sol.time_to_find) for sol in solver.solutions_cache
    ]

    improving_solutions = []
    best_obj = solver.solutions_cache[0].objective
    for i, sol in enumerate(solver.solutions_cache):
        if sol.objective < best_obj:
            best_obj = sol.objective
            improving_solutions.append(sol)

    improving_iterations = [sol.iteration for sol in improving_solutions]
    improving_objectives = [sol.objective for sol in improving_solutions]
    improving_times = [
        "{:.2f}".format(sol.time_to_find) for sol in improving_solutions
    ]

    # fig.update_layout(xaxis2= {'anchor': 'y', 'overlaying': 'x', 'side': 'top'})

    fig.add_trace(go.Scatter(x=iterations,
                            y=objectives,
                            name=name+" objective along iterations",
                            # marker_color="#636EFA"
                            ),
                row=1,
                col=1)
    fig.add_trace(go.Scatter(x=time_found,
                            y=objectives,
                            name=name+" objective along time",
                            # marker_color="#636EFA"
                            ),
                row=1,
                col=2)

    fig.add_trace(go.Scatter(x=improving_iterations,
                            y=improving_objectives,
                            mode='markers',
                            name=name+" improving iterations",
                            # marker_color="green"
                            ),
                row=1,
                col=1)
    fig.add_trace(go.Scatter(x=improving_times,
                            y=improving_objectives,
                            mode='markers',
                            name=name+" improving times",
                            # marker_color="green"
                            ),
                row=1,
                col=2)

    # fig.data[1].update(xaxis='x2')
    # fig.data[3].update(xaxis='x2')

    fig.update_xaxes(title_text="iteration", row=1, col=1)
    fig.update_xaxes(title_text="time", row=1, col=2)
    fig.update_yaxes(title_text="objective", row=1, col=1)

    fig.update_layout(height=600, width=1600, title_text="Objective Value as the LNS Progresses")
    return fig

In [34]:
fig = make_subplots(
        rows=1,
        cols=2,
        shared_yaxes=True,
    )
plot_solver_stats(relaxed_LNS, fig, "relaxed")
plot_solver_stats(normal_LNS, fig, "normal")
fig.show()